

> **Algoritmo que analisa o banco de dados do stackoverflow e analisa a qual tag pertence utilizando os algoritmos de classificação: Relevância Binária, Classificação em Cadeia e MLkNN**



# **Setup inicial do banco de dados**

In [ ]:
#importando biblioteca
import pandas as pd

#leitura do dataset
perguntas = pd.read_csv("stackoverflow_perguntas.csv")
#perguntas.head(10)

In [ ]:
#analisando quais tags temos no nosso base de dados
print(f'>> Temos nesse dataset {len(perguntas.Tags.unique())} tags.')

print('\n>> As tags presentes nesse dataset são:\n')
perguntas.Tags.unique()

>> Temos nesse dataset 37 tags.

>> As tags presentes nesse dataset são:



array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [ ]:
#descobrindo quais as labels que temos dentro da coluna Tag
lista_de_tags = list()

#exemplo que corta os elementos do vetor
#"node.js html".split()
for tags in perguntas.Tags.unique():
  for tag in tags.split():
    if tag not in lista_de_tags:
      lista_de_tags.append(tag)

print(">> Os labels presentes dentro das tags são:\n")
print(lista_de_tags)

>> Os labels presentes dentro das tags são:

['node.js', 'jquery', 'html', 'angular']


In [ ]:
#lendo se a nossa lista possui o label de node.hs
node_js = list()

for linha_tag in perguntas.Tags:
  if "node_js" in linha_tag:
    node_js.append(1)
  else:
    node_js.append(0)

#adicionando essa lista no dataset
#perguntas["node_js"] = node_js

#print('>> Após análise, a inserção da coluna que mapeia a presença de node.js em casa Tag:\n')
#perguntas.head()

In [ ]:
#funcao que mapeia a presença de cada label na lista de tags
#em seguida, cria uma coluna binária mapeando a existência

def nova_coluna(lista_de_tags, dataframe, nome_tags):
  for tag in lista_de_tags:
    coluna = list()
    for linha_tag in dataframe[nome_tags]:
      if tag in linha_tag:
        coluna.append(1)
      else:
        coluna.append(0)
    #adicionando ao dataframe
    dataframe[tag] = coluna

#inserindo as novas colunas no dataframe
nova_coluna(lista_de_tags, perguntas, "Tags")

#imprimindo para visualizar
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


In [ ]:
#para avançar é imprecindível ter todas as colunas dummies presentes em uma única coluna
#sendo assim, vamos juntar as colunas de Node.js, jquery, html e angular

#aprendendo a utilizar o zip para juntas colunas no dataframe
lista_1 = [1,2]
lista_2 = [5,4]
out = zip(lista_1, lista_2) 
#print(list(out))

#aplicando zip ao dataframe
lista_zip_tags = list(zip(perguntas[lista_de_tags[0]], 
                      perguntas[lista_de_tags[1]],
                      perguntas[lista_de_tags[2]],
                      perguntas[lista_de_tags[3]])
                      )

#adicionando essa lista no datagrame
perguntas['todas_tags'] = lista_zip_tags

#visualizando as modificações
perguntas.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
3822,Tem alguma maneira de enviar o blob pro servid...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3084,Realizo uma consulta com ajax em uma pagina ph...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3325,A função abaixo não me retorna o esperado. Eu...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3897,Bom dia! Não estou conseguindo deslizar para o...,html,0,0,1,0,"(0, 0, 1, 0)"
110,Estou montando um webapp com o framework JQuer...,jquery,0,1,0,0,"(0, 1, 0, 0)"
380,Tenho o seguinte código HTML: CODE Eu preci...,html,0,0,1,0,"(0, 0, 1, 0)"
3671,Criei um script básico para exibir/ocultar um ...,jquery,0,1,0,0,"(0, 1, 0, 0)"
1744,Como centralizar a div CODE que é criada de f...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3864,"Uma dúvida, porquê isso trabalha: CODE e is...",jquery,0,1,0,0,"(0, 1, 0, 0)"
5092,Bom dia Galera. Estou consumindo uma api exte...,node.js,1,0,0,0,"(1, 0, 0, 0)"




> **Dividindo nosso dataset em treino e teste**



In [ ]:
#criando os conjuntos de teste e treino

#biblioteca utilizada
from sklearn.model_selection import train_test_split

#X_train, X_test, Y_train, Y_test
perguntas_treino, perguntas_test, tags_treino, tags_test = train_test_split(perguntas.Perguntas,
                                                                            perguntas.todas_tags,
                                                                            test_size=0.2,
                                                                            random_state=123)
#test_size = % do tamanho do dataset que será utilizada para teste
#random_state = é apenas um valor para garantirmos sempre a mesma subdivisão


> **Conversão de linguagem humana para linguagem de máquina**



*Como o computador não entende o que significa a palavra "não" ou qualquer outra do nosso dataset, vamos utilizar a técnica TF-IDF. Ela atribui uma relevância para cada palavra nas perguntas.*

In [ ]:
#biblioteca utilizada para transformar as perguntas
from sklearn.feature_extraction.text import TfidfVectorizer

#realizando a operação de vetorizar
vetorizar = TfidfVectorizer(max_features=5000,max_df=0.85) #vetor com o tamanho de 5000 e eliminando palavras que aparecem acima de 85% em cada pergunta

In [ ]:
#aplicando nosso algoritmo para realizar a transformação
vetorizar.fit(perguntas.Perguntas) 

#realizando a conversao das perguntas de treino e teste para linguagem de máquina
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_test_tfidf = vetorizar.transform(perguntas_test)

#dimensoes dos nossos vetores
print('>> As dimensões das perguntas de treino são:\n')
print(perguntas_treino_tfidf.shape)
print('\n>> As dimensões das perguntas de teste são:\n')
print(perguntas_test_tfidf.shape)

>> As dimensões das perguntas de treino são:

(4326, 5000)

>> As dimensões das perguntas de teste são:

(1082, 5000)


# **Algoritmo de classificação: Relevância Binária**

*Esse algoritmo binário cria um modelo para cada categoria e classifica de acordo com 0(não presença) e 1(presença)*



> Métrica da qualidade do algoritmo: Extra-match (score)



In [ ]:
#vamos utilizar um modelo de classificação multilabel 
#chamado de relevância binária

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

#criando um classificador binário
regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)

#antes de treinar o modelo vamos ter que converter a ultima coluna em um array ou matrix binária
tags_treino_array = np.asarray(list(tags_treino)) #temos que usar o list() pois se não teremos uma lista de tuplas
tags_test_array = np.asarray(list(tags_test))
#print(type(tags_test_array))
#print(tags_treino_array)

#treinando o nosso modelo com os dados de treino
classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino_array)

#agora predizendo o modelo através dos dados de teste, o score trás a acurácia
resultado_onevsrest = classificador_onevsrest.score(perguntas_test_tfidf, tags_test_array)
print('Resultado da acurácia {0: .2f}%'.format(resultado_onevsrest*100))

#vale lembrar que para classificação multilabel, a acurácia não é uma boa métrica
#para medir a qualidade do nosso algoritmo. Isso acontece pois ela busca o exact match.

Resultado da acurácia  41.68%




> Métrica de qualidade: hamming loss



In [ ]:
from sklearn.metrics import hamming_loss

#predizendo nossas tags através das perguntas
previsao_onevsrest = classificador_onevsrest.predict(perguntas_test_tfidf)

#utilizando as tags de test e comparando com o valor previsto pelo classificador
hamming_loss = hamming_loss(tags_test_array, previsao_onevsrest)

#imprimindo os resultados
#vale lembrar que quanto mais próximo de zero, melhor é o nosso modelo
print('hamming_loss {0: .2f}'.format(hamming_loss))

hamming_loss  0.19


*Agora vamos analisar para ver se temos alguma correlação entre as variáveis.*

In [ ]:
#analisando a correlação entre as variáveis

perguntas.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


> **Implementando o Relevância Binária utilizando o skmultilearn**


In [46]:
!pip install scikit-multilearn

from skmultilearn.problem_transform import BinaryRelevance

classificador_br = BinaryRelevance(regressao_logistica)
classificador_br.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_br = classificador_br.score(perguntas_test_tfidf, tags_test_array)
previsao_br = classificador_br.predict(perguntas_test_tfidf)

#aplicando hamming loss
hamming_loss_br = hamming_loss(tags_test_array, previsao_br)

#imprimindo os resultados 
print("Hamming Loss {0: .2f}".format(hamming_loss_br))
print("Resultado {0: .2f}%".format(resultado_br*100))


Hamming Loss  0.19
Resultado  41.68%


# ***Algoritmo de classificação: Classificação em Cadeia***

*Esse algoritmo cria um modelo para cada tag. Esse algoritmo cria um modelo que analisa todas as 4 colunas*

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.metrics import hamming_loss

#criando meu classificador multilabel
classificador_cadeia = ClassifierChain(regressao_logistica)

#treinando meus dados com esse classificador
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)

#calculando a acurácia
resultado_cadeia = classificador_cadeia.score(perguntas_test_tfidf, tags_test_array)

#calculando o hamming loss
previsao_cadeia = classificador_cadeia.predict(perguntas_test_tfidf)
hamming_loss2 = hamming_loss(tags_test_array, previsao_cadeia)

#imprimindo os resultados 
print("Hamming Loss {0: .2f}".format(hamming_loss2))
print("Resultado {0: .2f}%".format(resultado_cadeia*100))

Hamming Loss  0.21
Resultado  49.82%


# **Algoritmo de classificação: Multi-label KNN (MLkNN)**

> **Utilizando o classificador MLKNN**


In [ ]:
from skmultilearn.adapt import MLkNN

classificador_mlknn = MLkNN()
classificador_mlknn.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_mlknn = classificador_mlknn.score(perguntas_test_tfidf, tags_test_array)
previsao_mlknn = classificador_mlknn.predict(perguntas_test_tfidf)

#aplicando hamming loss
hamming_loss_mlknn = hamming_loss(tags_test_array, previsao_mlknn)

#imprimindo os resultados 
print("Hamming Loss {0: .2f}".format(hamming_loss_mlknn))
print("Resultado {0: .2f}%".format(resultado_mlknn*100))


Hamming Loss  0.25
Resultado  32.53%
